# ADS2 - Assignment 1 - Data Handling and Processing with PySpark

**STUDENT NAME -** Vegu Mahesh Babu

**STUDENT ID -** 22034351

In this assignment, you will be analysing the popularity of films and TV shows on the streaming platform, Netflix. Using your knowledge of PySpark DataFrames and Spark SQL, you will produce a number of "downstream" data products to analyse trends in global streaming habits.

Download the dataset from this [Kaggle](https://www.kaggle.com/dhruvildave/netflix-top-10-tv-shows-and-films) page. A copy of the `all_weeks_countries.csv` file is also available on the canvas page for this assignment.

Your task is to load in the data and produce a number of "downstream" data products and plots as described below.

The PySpark installation and setup is provided below for conveinience.

IMPORTANT: DO NOT EDIT OR REMOVE THE COMMENT TAGS IN THE CODE CELLS

In [2]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

Mounted at /content/drive


In [4]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [5]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [6]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [7]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.

netflixcsvpath = "file:///content/drive/MyDrive/all-weeks-countries.csv"

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [8]:

# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [9]:
# CodeGrade Tag Ex1a

### Display the table and its schema
netflixDF.show()
netflixDF.printSchema()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                null|                         1|
|   Argentina|          AR|2022-08

In [10]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF
filmsDF = netflixDF.filter(netflixDF.category == 'Films')
tvDF = netflixDF.filter(netflixDF.category == 'TV')


filmsDF.show()
tvDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|        null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|        null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|        null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|        null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...| 

In [11]:
# CodeGrade Tag Ex1c

### Drop the 'season_title' column from the Films DataFrame, display the table
filmsDF = filmsDF.drop('season_title')

filmsDF.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

In [12]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.

tvDF = tvDF.withColumn('season_title_null', F.isnull(tvDF['season_title']))

null_condition = F.isnull(tvDF['season_title'])
tvDF = tvDF.withColumn('season_title', F.when(null_condition, tvDF['show_title']).otherwise(tvDF['season_title']))

count_season_titles = tvDF.select('season_title').distinct().count()
count_df = spark.createDataFrame([(count_season_titles,)], ['season_title'])

tvDF=tvDF.drop('season_title_null')
tvDF.show()
count_df.show()

+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          2|        Another Self|Another Self: Sea...|                         4|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          4|            Manifest|  Manifest: Season 1|                         6|
|   Argentina|          AR|2022-08

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [13]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.
STDF = tvDF.where((tvDF.show_title == "Stranger Things") & (tvDF.country_name == "United Kingdom"))



### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.
STWeeks = STDF.groupBy("show_title").agg(F.count_distinct("week").alias("STWeeks"))

STDF.show(10)
STWeeks.show()


+--------------+------------+-------------------+--------+-----------+---------------+-----------------+--------------------------+
|  country_name|country_iso2|               week|category|weekly_rank|     show_title|     season_title|cumulative_weeks_in_top_10|
+--------------+------------+-------------------+--------+-----------+---------------+-----------------+--------------------------+
|United Kingdom|          GB|2022-08-21 00:00:00|      TV|          3|Stranger Things|Stranger Things 4|                        13|
|United Kingdom|          GB|2022-08-14 00:00:00|      TV|          3|Stranger Things|Stranger Things 4|                        12|
|United Kingdom|          GB|2022-08-14 00:00:00|      TV|         10|Stranger Things|Stranger Things 3|                        12|
|United Kingdom|          GB|2022-08-07 00:00:00|      TV|          2|Stranger Things|Stranger Things 4|                        11|
|United Kingdom|          GB|2022-08-07 00:00:00|      TV|          9|Strang

# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [14]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25


Top25 = tvDF.filter(tvDF['country_name'] == 'United Kingdom')\
       .groupBy('season_title')\
       .agg(F.max('cumulative_weeks_in_top_10').alias('total_weeks'))\
       .orderBy(F.desc('total_weeks'))\
       .limit(25)

Top25.show(25)


+--------------------+-----------+
|        season_title|total_weeks|
+--------------------+-----------+
|   Stranger Things 4|         13|
|     Ozark: Season 4|         12|
|   Stranger Things 3|         12|
|   Stranger Things 2|         11|
|Squid Game: Season 1|         10|
|     Stranger Things|         10|
|Better Call Saul:...|         10|
|Bridgerton: Season 2|          9|
|Maid: Limited Series|          9|
|PAW Patrol: Season 6|          9|
|Inventing Anna: L...|          8|
| Money Heist: Part 5|          7|
|  Sex/Life: Season 1|          7|
|PIECES OF HER: Se...|          7|
|Clickbait: Limite...|          6|
|Sex Education: Se...|          6|
|Virgin River: Sea...|          6|
|Young Sheldon: : ...|          6|
|Young Sheldon: : ...|          6|
|The Lincoln Lawye...|          6|
|       You: Season 3|          6|
|PAW Patrol: Season 4|          6|
|The Umbrella Acad...|          5|
|Love Is Blind: Se...|          5|
|The Witcher: Seas...|          5|
+-------------------

# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [15]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10

young_sheldon = tvDF.filter(tvDF["show_title"] == "Young Sheldon")

max_weeks_per_season_country = (
    young_sheldon
    .groupBy("season_title", "country_name")
    .agg(F.max("cumulative_weeks_in_top_10").alias("max_weeks_in_top_10"))
    .sort(F.desc("max_weeks_in_top_10"))
)

max_weeks_country =max_weeks_per_season_country.groupBy("season_title").agg(F.first("country_name").alias("first(country_name)")).orderBy("season_title")
max_weeks_country.show(truncate=False)

+-------------------------+-------------------+
|season_title             |first(country_name)|
+-------------------------+-------------------+
|Young Sheldon: : Season 1|Canada             |
|Young Sheldon: : Season 2|United Kingdom     |
|Young Sheldon: : Season 3|United Kingdom     |
|Young Sheldon: Season 4  |New Zealand        |
+-------------------------+-------------------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.

In [24]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.

filtered_films = netflixDF.filter(netflixDF['category'] == 'Films')

maxweeks = filtered_films.groupBy('country_name').agg(
    F.max('cumulative_weeks_in_top_10').alias('max_weeks'),
    F.sum('cumulative_weeks_in_top_10').alias('sum_weeks')
)

alias = filtered_films.alias('films')
max_weeks_alias = maxweeks.alias('max_weeks')

result_films = alias.join(
    max_weeks_alias,
    (alias['country_name'] == max_weeks_alias['country_name']) &(alias['cumulative_weeks_in_top_10'] == max_weeks_alias['max_weeks']),
    'inner').select(alias['country_name'], alias['show_title'], alias['cumulative_weeks_in_top_10'])

result_films = result_films.dropDuplicates(['country_name'])
result_films.select('country_name','show_title').show(20,truncate=False)
result_films = result_films.sort('cumulative_weeks_in_top_10', ascending=False)


+------------------+-------------------------------------+
|country_name      |show_title                           |
+------------------+-------------------------------------+
|Argentina         |Sonic the Hedgehog                   |
|Australia         |Back to the Outback                  |
|Austria           |Harry Potter and the Sorcerer's Stone|
|Bahamas           |A Madea Homecoming                   |
|Bahrain           |Red Notice                           |
|Bangladesh        |Sooryavanshi                         |
|Belgium           |The Adam Project                     |
|Bolivia           |Shrek                                |
|Brazil            |Hotel Transylvania 3: Summer Vacation|
|Bulgaria          |Red Notice                           |
|Canada            |The Adam Project                     |
|Chile             |Sonic the Hedgehog                   |
|Colombia          |Red Notice                           |
|Costa Rica        |Sonic the Hedgehog                  

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [25]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spens in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"

netflixDF = netflixDF.withColumn(
    "weeks_at_1",
    F.when(F.col("weekly_rank") == 1, 1).otherwise(0)
).alias("weeks_at_1")

# Show the resulting DataFrame
netflixDF.select("show_title", "weekly_rank", "weeks_at_1").show()

+--------------------+-----------+----------+
|          show_title|weekly_rank|weeks_at_1|
+--------------------+-----------+----------+
|      Look Both Ways|          1|         1|
|           Day Shift|          2|         0|
|Bank Robbers: The...|          3|         0|
|   The Next 365 Days|          4|         0|
|The Angry Birds M...|          5|         0|
|       Purple Hearts|          6|         0|
|      River Runs Red|          7|         0|
|                1917|          8|         0|
|  Code Name: Emperor|          9|         0|
|           Royalteen|         10|         0|
| Pasión de Gavilanes|          1|         1|
|        Another Self|          2|         0|
| Pasión de Gavilanes|          3|         0|
|            Manifest|          4|         0|
|         The Sandman|          5|         0|
|Extraordinary Att...|          6|         0|
|           High Heat|          7|         0|
|            Manifest|          8|         0|
|            Manifest|          9|

In [24]:
 # CodeGrade Tag Ex6b

### Group by country name and sow title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.

result_df = netflixDF.groupBy("country_name", "show_title").agg(
    F.sum("weeks_at_1").alias("total_weeks_at_1")
).sort(F.desc("total_weeks_at_1"))

result_df.show()

+------------------+--------------------+----------------+
|      country_name|          show_title|total_weeks_at_1|
+------------------+--------------------+----------------+
|          Colombia|Newly Rich, Newly...|              18|
|           Bolivia|Yo soy Betty, la fea|              17|
|         Venezuela|Café con aroma de...|              13|
|       El Salvador|Café con aroma de...|              13|
|          Honduras|   Pedro el escamoso|              12|
|       El Salvador|   Pedro el escamoso|              12|
|         Guatemala|   Pedro el escamoso|              12|
|         Argentina|Café con aroma de...|              12|
|          Honduras|Café con aroma de...|              12|
|Dominican Republic|Café con aroma de...|              12|
|         Nicaragua|   Pedro el escamoso|              12|
|         Nicaragua|Café con aroma de...|              12|
|             Chile|Café con aroma de...|              11|
|         Guatemala|Café con aroma de...|              1

In [25]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.

top_df = netflixDF.groupBy("country_name", "show_title").agg(
    F.sum("weeks_at_1").alias("total_weeks_at_1")
)

result_df = top_df.groupBy("country_name").agg(
    F.first("show_title").alias("show_title"),
    F.max("total_weeks_at_1").alias("total_weeks_at_1")
)

result_df.show()

+------------------+--------------------+----------------+
|      country_name|          show_title|total_weeks_at_1|
+------------------+--------------------+----------------+
|         Argentina|       In Good Hands|              12|
|         Australia|           Bloodshot|               7|
|           Austria|            For Life|               7|
|           Bahamas|        Virgin River|               4|
|           Bahrain|          Spiderhead|               8|
|        Bangladesh|             Nikamma|               8|
|           Belgium|        Heart Parade|               7|
|           Bolivia|Men in Black: Int...|              17|
|            Brazil|Cúmplices de um R...|               7|
|          Bulgaria|G.I. Joe: The Ris...|               9|
|            Canada|   Indecent Proposal|               7|
|             Chile|            Intimacy|              11|
|          Colombia|               Lulli|              18|
|        Costa Rica|  Dangerous Liaisons|              1